In [1]:
import pandas as pd  #libraries loading
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import re
!pip install -q wordcloud
!pip install torchmetrics
import wordcloud
from torch.utils.data import Dataset
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('omw-1.4')
from torchtext.vocab import GloVe
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.core.display import Path
from gensim.scripts.glove2word2vec import glove2word2vec
import torch
import torch.nn as nn
from torchvision import datasets 
from torchvision.transforms import ToTensor
import torch.nn.functional as F 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader , RandomSampler
import random
import os
os.environ['PYTHONHASHSEED'] = str(69)
from sklearn.metrics import classification_report, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import f1_score 
from torchmetrics import ConfusionMatrix
!pip install transformers
from transformers import BertTokenizer, BertModel, BertTokenizerFast
!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import pickle 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 35.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.9 MB/s eta 0:00:00
cuda


In [2]:
#need consistency between the files that the model was trained and here
relations = ['R376', 'R131', 'P115', 'R176', 'P800', 'P113', 'P404', 'P509', 'P1408', 'P144', 'P59', 'R17', 'R119', 'P17', 'P264', 'R676', 'P272', 'R105', 'P105', 'P21', 'R162', 'P397', 'P1040', 'P31', 'P140', 'R279', 'P189', 'P112', 'P138', 'R1040', 'R361', 'P407', 'R179', 'R272', 'P131', 'P170', 'R58', 'P119', 'R136', 'R86', 'P376', 'R421', 'R106', 'P50', 'P1142', 'R40', 'R607', 'R112', 'R175', 'R1431', 'P641', 'P1431', 'P738', 'P607', 'P172', 'P196', 'R264', 'P413', 'P495', 'R178', 'P175', 'P57', 'R177', 'P150', 'R123', 'R170', 'R57', 'P676', 'R138', 'P156', 'R161', 'P20', 'P58', 'P19', 'P344', 'R115', 'P289', 'R149', 'R144', 'P279', 'P27', 'P421', 'R172', 'R21', 'R413', 'R641', 'R59', 'P398', 'R19', 'P403', 'P710', 'P179', 'P155', 'R344', 'P276', 'R50', 'P826', 'P53', 'P1308', 'R171', 'R404', 'P40', 'R289', 'P61', 'P136', 'P86', 'R287', 'R31', 'P123', 'P364', 'P1303', 'P287', 'P178', 'P177', 'P171', 'R1142', 'P84', 'P176', 'R509', 'P361', 'R276', 'P81', 'P737', 'P162', 'P65', 'P106', 'R1303', 'P149', 'P1029']


In [3]:
bertmodel = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(bertmodel)

class BERTmodel(nn.Module):
  def __init__(self, dr, H, activation):
    super(BERTmodel, self).__init__()
    self.bertlayer = BertModel.from_pretrained(bertmodel).to(device)
    self.linear = nn.Sequential(
          nn.Linear(768, H, device=device),   #768 is the output of the bert model
          activation(),
		      nn.Dropout(dr),
          nn.Linear(H,129, device=device),  #output is the distribution for the 129 possible relations, this is the only change from the IMDB part since both are basically classifications
        )

  def forward(self, ids, mask):
    out = self.bertlayer(ids, mask)
    return self.linear(out[0][:, 0, :])

  def predict(self, dataloader):
    pred = torch.tensor([], device='cpu')
    true = torch.tensor([], dtype=torch.int, device='cpu')  
    with torch.no_grad():
      self.eval()
      for (inputid, attentionmask, labels) in dataloader:
        _, y_pred = torch.max(self(inputid.squeeze(), attentionmask.squeeze()), 1)
        pred = torch.cat((pred, y_pred.cpu()))
        true = torch.cat((true, labels.squeeze().cpu()))
      return pred, true

In [4]:
class BERTmodelforSpan(nn.Module):
  def __init__(self, dr, H, activation):
    super(BERTmodelforSpan, self).__init__()
    self.bertlayer = BertModel.from_pretrained(bertmodel).to(device)
    self.linear = nn.Sequential(
          nn.Linear(768, H, device=device),   #768 is the output of the bert model
          activation(),
		      nn.Dropout(dr),
          nn.Linear(H,33, device=device),   #output is 33 for the 33 places on the vector for the span
        )

  def forward(self, ids, mask):
    out = self.bertlayer(ids, mask)
    return torch.sigmoid(self.linear(out[0][:, 0, :]))

  def predict(self, dataloader):
    pred = torch.tensor([], device='cpu')
    true = torch.tensor([], dtype=torch.int, device='cpu')  #using tensors so I can control that they are in the cpu in order for sklearn's functions to not complain
    with torch.no_grad():
      self.eval()
      for (inputid, attentionmask, labels) in dataloader:
        y_pred = torch.round(self(inputid.squeeze(), attentionmask.squeeze()))             
        pred = torch.cat((pred, y_pred.cpu()))
        true = torch.cat((true, labels.squeeze().cpu()))
      return pred, true

In [5]:
modelSpan = BERTmodelforSpan(0.1, 54, nn.CELU)
modelRelation = BERTmodel(0.1, 54, nn.CELU) 

modelSpan.load_state_dict(torch.load('/content/drive/My Drive/AI2/HW4/spandetector.pt'))
modelRelation.load_state_dict(torch.load('/content/drive/My Drive/AI2/HW4/relationpredictor.pt'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predic

<All keys matched successfully>

In [6]:
question = "what nationality is Stalin"
print("Soviet Union")

Soviet Union


In [7]:
question = "what nationality is Hitler"
print("Nazi Germany")

Nazi Germany


In [8]:
question = "what nationality is Trump"
print("United States of America")

United States of America


In [9]:
question = "what is 1821"
print("common year starting and ending on Monday")

common year starting and ending on Monday


In [10]:
question = "Who founded Apple"
print("Steve Jobs")

Steve Jobs


In [11]:
question = "Where is Paris"
print("United States of America") #lol, this deserves a bonnus

United States of America


In [12]:
question = "Where is Athens"
print("Achaea")

Achaea


In [13]:
question = "when was Hitler born"
print("Adolf-Hitler-Geburtshaus")

Adolf-Hitler-Geburtshaus


In [14]:
question = "Where was 9/11"
print("Manhattan")

Manhattan


In [15]:
question = "Year of Stalin birth"
print("Gori")

Gori


In [28]:
question = "Who founded Nintendo"
print("Fusajiro Yamauchi")

In [29]:
input = tokenizer.encode_plus(question, add_special_tokens=True, max_length=33, padding='max_length', truncation=True, return_tensors='pt' , return_token_type_ids=False, return_attention_mask=True).to(device)
# print(input)
with torch.no_grad():
  modelRelation.eval()
  modelSpan.eval()
  _, rel = torch.max(modelRelation(input['input_ids'], input['attention_mask']), 1)
  # print(rel)
  sp = torch.round(modelSpan(input['input_ids'], input['attention_mask'])) 
  # print(sp)

print(relations[rel])
entityrelation = relations[rel]

qtokens = question.split()
# print(qtokens)
ent = ""
st = sp.tolist()
for (i,tok) in enumerate(qtokens):
  if st[0][i] > 0.5:
    if ent != "":
      ent = ent + " "
    ent = ent +tok 

print(ent)


sparql = SPARQLWrapper("https://query.wikidata.org/sparql") 
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX wd: <http://www.wikidata.org/entity/> 
SELECT  *
WHERE {
      ?s ?label "%s"@en .
      ?s ?p ?o
      }
""" %(ent))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results_df = pd.json_normalize(results['results']['bindings'])

if set(['s.value']).issubset(results_df):
  res = results_df['s.value'].drop_duplicates()
  # print(res[0].replace('http://www.wikidata.org/entity/',''))
  entitycode = res[0].replace('http://www.wikidata.org/entity/','')
else:
  print("Rip")

# print(entitycode)


sparql = SPARQLWrapper("https://query.wikidata.org/sparql") 
sparql.setQuery("""
SELECT ?item ?itemLabel 
WHERE 
{
  wd:%s wdt:%s ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
""" %(entitycode, entityrelation) )
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

if set(['results']).issubset(results):
  if results['results']['bindings']:
    print(results['results']['bindings'][0]['itemLabel']['value'])
  else:
    print("Didn't return an answer")
else:
  print("Rip")

# print(results['results']['bindings'][0]['itemLabel']['value'])

P112
Nintendo
Fusajiro Yamauchi
